#### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import datetime
import pandas_datareader.data as web #извлекает данные из различных интернет-источников в фрейм данных pandas
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from plotly.subplots import make_subplots

/Users/anastasiafirsova/repo/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Начальные значения

In [2]:
money = 1000000
start = datetime.datetime(2015, 1, 1)
finish = datetime.datetime(2020, 12, 31)
short_window = 30
long_window = 90
stop_loss = 0.05  # 5% от цены покупки
ticker = 'MA'

In [3]:
start_str = start.strftime('%Y-%m-%d') # в условии требовалась дата в таком формате
end_str = finish.strftime('%Y-%m-%d')

#### Используем библиотеку yahoo finance, она позволит нам взять сразу все данные по акциям, сайт MOEX даёт только первые 100 записей

In [4]:
#pip install yfinance

In [5]:
import yfinance as yf

#### Загружаем данные с помощью библиотек по данному тикеру

In [6]:
info = yf.download(ticker, start=start_str, end=end_str)
info.index = pd.to_datetime(info.index)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [7]:
info

Price,Close,High,Low,Open,Volume
Ticker,MA,MA,MA,MA,MA
Date,,,,,
2015-01-02,80.319992,81.894894,79.607534,81.257434,4505800
2015-01-05,78.060707,80.010586,77.854476,79.954342,6441400
2015-01-06,77.892006,78.538843,76.682713,78.426354,7690000
2015-01-07,79.103638,79.159990,78.023499,78.990925,5846800
2015-01-08,80.334030,80.418566,79.864405,80.014689,5174200
...,...,...,...,...,...
2020-12-23,320.894684,326.723265,320.699735,325.670621,2760600
2020-12-24,327.493317,327.853944,321.986367,322.278759,1308900


In [8]:
info = info[['Open', 'High', 'Low', 'Close']] #оставляем нужные столбцы

In [9]:
info

Price,Open,High,Low,Close
Ticker,MA,MA,MA,MA
Date,,,,
2015-01-02,81.257434,81.894894,79.607534,80.319992
2015-01-05,79.954342,80.010586,77.854476,78.060707
2015-01-06,78.426354,78.538843,76.682713,77.892006
2015-01-07,78.990925,79.159990,78.023499,79.103638
2015-01-08,80.014689,80.418566,79.864405,80.334030
...,...,...,...,...
2020-12-23,325.670621,326.723265,320.699735,320.894684
2020-12-24,322.278759,327.853944,321.986367,327.493317


In [10]:
info.columns = info.columns.droplevel(1) #приводим в нужный вид данные

In [11]:
info

Price,Open,High,Low,Close
Date,,,,
2015-01-02,81.257434,81.894894,79.607534,80.319992
2015-01-05,79.954342,80.010586,77.854476,78.060707
2015-01-06,78.426354,78.538843,76.682713,77.892006
2015-01-07,78.990925,79.159990,78.023499,79.103638
2015-01-08,80.014689,80.418566,79.864405,80.334030
...,...,...,...,...
2020-12-23,325.670621,326.723265,320.699735,320.894684
2020-12-24,322.278759,327.853944,321.986367,327.493317
2020-12-28,329.306204,336.840510,328.341278,335.748871


In [12]:
info.columns.name = None  # удаляем имя 'Price' у заголовков столбцов
info = info.reset_index()
info= info.rename(columns = {'index':'Date'})

In [13]:
info

,Date,Open,High,Low,Close
0,2015-01-02,81.257434,81.894894,79.607534,80.319992
1,2015-01-05,79.954342,80.010586,77.854476,78.060707
2,2015-01-06,78.426354,78.538843,76.682713,77.892006
3,2015-01-07,78.990925,79.159990,78.023499,79.103638
4,2015-01-08,80.014689,80.418566,79.864405,80.334030
...,...,...,...,...,...
1505,2020-12-23,325.670621,326.723265,320.699735,320.894684
1506,2020-12-24,322.278759,327.853944,321.986367,327.493317
1507,2020-12-28,329.306204,336.840510,328.341278,335.748871
1508,2020-12-29,338.975001,339.920444,336.031471,337.883362


#### Скользящие окна рассчитываются как простое среднее

In [14]:
info['short_sliding_window'] = info['Close'].rolling(short_window).mean() #короткое скользящее окно
info['long_sliding_window'] = info['Close'].rolling(long_window).mean() #длинное скользящее окно

In [15]:
info

,Date,Open,High,Low,Close,short_sliding_window,long_sliding_window
0,2015-01-02,81.257434,81.894894,79.607534,80.319992,NaN,NaN
1,2015-01-05,79.954342,80.010586,77.854476,78.060707,NaN,NaN
2,2015-01-06,78.426354,78.538843,76.682713,77.892006,NaN,NaN
3,2015-01-07,78.990925,79.159990,78.023499,79.103638,NaN,NaN
4,2015-01-08,80.014689,80.418566,79.864405,80.334030,NaN,NaN
...,...,...,...,...,...,...,...
1505,2020-12-23,325.670621,326.723265,320.699735,320.894684,326.775588,325.773410
1506,2020-12-24,322.278759,327.853944,321.986367,327.493317,326.892873,325.831727
1507,2020-12-28,329.306204,336.840510,328.341278,335.748871,327.418554,325.966631
1508,2020-12-29,338.975001,339.920444,336.031471,337.883362,327.796403,326.072030


#### Теперь нужно добавить сигналы к покупке и продаже 

##### sig_buy (покупка):
##### Короткая скользящая средняя пересекает длинную снизу вверх.
##### sig_sale (продажа):
##### Короткая скользящая средняя пересекает длинную сверху вниз.

In [16]:
info['sig_buy'] = np.where((info['short_sliding_window'].shift(1) <= info['long_sliding_window'].shift(1)) &
                         (info['short_sliding_window'] > info['long_sliding_window']), info['Close'], np.nan)
info['sig_sale'] = np.where((info['short_sliding_window'].shift(1) >= info['long_sliding_window'].shift(1)) &
                          (info['short_sliding_window'] < info['long_sliding_window']), info['Close'], np.nan)

In [17]:
info

,Date,Open,High,Low,Close,short_sliding_window,long_sliding_window,sig_buy,sig_sale
0,2015-01-02,81.257434,81.894894,79.607534,80.319992,NaN,NaN,NaN,NaN
1,2015-01-05,79.954342,80.010586,77.854476,78.060707,NaN,NaN,NaN,NaN
2,2015-01-06,78.426354,78.538843,76.682713,77.892006,NaN,NaN,NaN,NaN
3,2015-01-07,78.990925,79.159990,78.023499,79.103638,NaN,NaN,NaN,NaN
4,2015-01-08,80.014689,80.418566,79.864405,80.334030,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1505,2020-12-23,325.670621,326.723265,320.699735,320.894684,326.775588,325.773410,NaN,NaN
1506,2020-12-24,322.278759,327.853944,321.986367,327.493317,326.892873,325.831727,NaN,NaN
1507,2020-12-28,329.306204,336.840510,328.341278,335.748871,327.418554,325.966631,NaN,NaN
1508,2020-12-29,338.975001,339.920444,336.031471,337.883362,327.796403,326.072030,NaN,NaN


#### Теперь нужно сделать такой датафрейм, который будет подходить для отчёта

In [18]:
report = pd.DataFrame(columns = ['date', 'signal', 'num_shares', 'share_price', 'share_value', 'cash'])

In [19]:
report #сформировали датафрейм нужного вида

,date,signal,num_shares,share_price,share_value,cash


#### Теперь необходимо перейти к торговле, сделаем цикл, с помощью которого будет заполнять наш отчёт

Сначала определим переменные, которые будут использоваться в цикле

In [20]:
num_shares = 0 #количество акций
share_value = 0 #текущая стоимость портфеля
cash = money #деньги

In [21]:
# Предыдущие значения, которые нужны для реализации цикла
last_num_shares = 0 
last_share_value = 0
last_money = cash
last_price_buy = 0

In [22]:
for i in range(len(info)):
    #теперь введем переменную, которая будет содержать строку
    row = info.iloc[i]

    #Теперь нужно определить стоп-лосс
    #Стоп-лосс сработает только, если у нас есть акции, то есть num_shares>0
    if num_shares>0:
        #определяем сам стоп-лосс 5% от покупки
        stop_loss_price = last_price_buy*(1 - stop_loss)
        #условие стоп-лосса
        if row['Low'] < stop_loss_price:
            #срабатывает стоп-лосс, продаём всё
            #Продажа происходит по цене стоп-лосса, даже если фактическая цена ниже.
            sell_price = stop_loss_price
            cash = round(cash+sell_price*num_shares,2) #подсчитали деньги от продажи
            num_shares = 0 #при стоп-лоссе продаем все акции
            share_value = 0 #так как продали все акции
            #добавляем данную информацию в отчёт
            new_row_report = pd.DataFrame({'date': [row['Date']], 'signal': ['stop-loss'], 
                       'num_shares': [last_num_shares], 'share_price': [sell_price],
                       'share_value': [last_share_value], 'cash': [cash]})
            report = pd.concat([report, new_row_report], ignore_index=True)
            #обновляем предыдущие значения
            last_num_shares = num_shares
            last_share_value = share_value
            last_money = cash
    #сигнал к покупке акций
    if row['sig_buy'] > 0:
        #записываем сигнал в отчёт
        new_row_report = pd.DataFrame({'date': [row['Date']], 'signal': ['sig_buy'], 
                       'num_shares': [last_num_shares], 'share_price': [row['Open']],
                       'share_value': [last_share_value], 'cash': [last_money]})
        report = pd.concat([report, new_row_report], ignore_index=True)
        #нужно, чтобы следующий день существовал
        if i+1 < len(info):
            #покупаем
            row_next_day = info.iloc[i+1] #строка следующий день
            share_price = row_next_day['Open'] #цена, по которой купили
            num_shares = cash // share_price #покупаем максимально возможное количество акций
            cash = round(cash - share_price*num_shares, 2) # оставшийся бюджет
            share_value = round(num_shares*share_price,2) #стоимость всех акций, которыыми мы владеем
            #записываем данные в отчёт
            new_row_report = pd.DataFrame({'date': [row_next_day['Date']], 'signal': ['buy'], 
                       'num_shares': [num_shares], 'share_price': [share_price],
                       'share_value': [share_value], 'cash': [cash]})
            report = pd.concat([report, new_row_report], ignore_index=True)
            #обновляем предыдущие значения
            last_num_shares = num_shares
            last_share_value = share_value
            last_money = cash
            last_price_buy = share_price
    #сигнал к продаже акций
    if row['sig_sale'] > 0 and last_num_shares > 0: #количество акций не может быть равно 0, так как будет нечего продавать
        #записываем сигнал в отчёт
        new_row_report = pd.DataFrame({'date': [row['Date']], 'signal': ['sig_sale'], 
                       'num_shares': [last_num_shares], 'share_price': [row['Open']],
                       'share_value': [last_share_value], 'cash': [last_money]})
        report = pd.concat([report, new_row_report], ignore_index=True)
        #нужно, чтобы следующий день существовал
        if  i+1 < len(info): 
            #продаем
            row_next_day = info.iloc[i+1] #строка следующий день
            share_price_sell = row_next_day['Open'] #цена, по которой продаём
            cash = round(cash+share_price_sell*last_num_shares,2) #бюджет после продажи
            num_shares = last_num_shares #продаём все акции
            share_value = 0 #продали акции
            #записываем данные в отчёт
            new_row_report = pd.DataFrame({'date': [row_next_day['Date']], 'signal': ['sale'], 
                       'num_shares': [num_shares], 'share_price': [share_price_sell],
                       'share_value': [share_value], 'cash': [cash]})
            report = pd.concat([report, new_row_report], ignore_index=True)
            #так как продали все акции, значит, их количество равно 0
            num_shares = 0
            #обновляем предыдущие значения
            last_num_shares = num_shares
            last_share_value = share_value
            last_money = cash
            last_price_buy = share_price

/var/folders/jd/3vq2txmx28z_qq2nx07_dy9r0000gn/T/ipykernel_14361/2073696136.py:33: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [23]:
report['num_shares'] = report['num_shares'].astype(int) #делаем количество целым числом

In [24]:
report

,date,signal,num_shares,share_price,share_value,cash
0,2015-10-27,sig_buy,0,93.804036,0,1000000
1,2015-10-28,buy,10727,93.218649,999956.45,43.55
2,2016-01-04,stop-loss,10727,88.557716,999956.45,950002.17
3,2016-04-11,sig_buy,0,89.034193,0,950002.17
4,2016-04-12,buy,10688,88.882514,949976.31,25.86
5,2016-06-27,stop-loss,10688,84.438388,949976.31,902503.35
6,2016-08-23,sig_buy,0,91.202374,0,902503.35
7,2016-08-24,buy,9865,91.477865,902429.14,74.21
8,2018-11-05,sig_sale,9865,191.082813,902429.14,74.21
9,2018-11-06,sale,9865,191.372019,0,1887959.18


#### Получаем последнее состояние портфеля

In [25]:
last_row = report.iloc[-1]

#### Требование: 
Вывести итоговую стоимость портфеля на момент последней сделки (sale или stop-loss).

Важно!
Если в конце периода открыта позиция (есть акции), прибыль/убыток считается по последней сделке.

#### Найдем последнюю сделку sale или stop-loss

In [26]:
last_sale = report[(report['signal'] == 'sale') | (report['signal'] == 'stop-loss')].tail(1)

In [27]:
if last_row['signal'] == 'buy': # то есть мы купили акции и в конце периода у нас открыта позиция
    final_value = round(last_row['num_shares']*last_row['share_price']+last_row['cash'],2)
    percentage_value = round(((final_value-1000000)/1000000),2)
    print(f'Итоговая стоимость портфеля: {final_value}')
    print(f'Процент прибыли/убытка: {percentage_value} %')
else: # иначе выводим стоимость на момент последней сделки типа 'sale' или 'loss'
    final_value = round(last_sale['num_shares'].iloc[0]*last_sale['share_price'].iloc[0],2)+last_sale['cash'].iloc[0]
    percentage_value = round(((final_value-1000000)/1000000),2)
    print(f'Итоговая стоимость портфеля: {final_value}')
    print(f'Процент прибыли/убытка: {percentage_value} %')

Итоговая стоимость портфеля: 2166878.21
Процент прибыли/убытка: 1.17 %


#### Визуализация

#### Для построения нижнего окна необходимо для начала сделать датафрейм, где будет зафиксировано , когда была покупка, а когда продажа

In [28]:
for_graph_down = info.copy()  #создаем копию, чтобы избежать предупреждений
for_graph_down['position'] = 0
current_position = 0 #для цикла показатель, с помощью которого мы будем заполнять датафрейм. Благодаря нему мы отслеживаем, сменился ли статус нашего бюджета (появились там акции или нет)

#### Далее с помощью цикла заполняем датафрейм

In [29]:
for i in range(1, len(info)):
    if pd.notna(for_graph_down['sig_buy'].iloc[i]) and current_position == 0: #есть сигнал покупки, который демонстрирует нам, что мы покупаем акции
        current_position = 1
    elif pd.notna(for_graph_down['sig_sale'].iloc[i]) and current_position == 1: #продажа - акций у нас нет
        current_position = 0
    for_graph_down.at[i, 'position'] = current_position  

#### Для начала определяем график OHLC

In [30]:
trace_ohlc = go.Ohlc(x=info['Date'],
                     open=info['Open'],
                     high=info['High'],
                     low=info['Low'],
                     close=info['Close'],
                     name='OHLC')

#### Определяем графики скользящих средних

In [31]:
line_short_window = go.Scatter(x=info['Date'],
                                y=info['short_sliding_window'],
                                mode='lines',
                                name='Короткое скользящее среднее',
                                line=dict(color='blue', width=1))
line_long_window = go.Scatter(x=info['Date'],
                               y=info['long_sliding_window'],
                               mode='lines',
                               name='Длинное скользящее среднее',
                               line=dict(color='orange', width=1))

#### Определяем графики сигналов

In [32]:
figure_sig_buy = go.Scatter(x=info[info['sig_buy'].notnull()]['Date'],
                           y=info['sig_buy'].dropna(),
                           mode='markers',
                           marker=dict(symbol='triangle-up', color='green', size=15),
                           name='Покупка')
figure_sig_sale = go.Scatter(x=info[info['sig_sale'].notnull()]['Date'],
                            y=info['sig_sale'].dropna(),
                            mode='markers',
                            marker=dict(symbol='triangle-down', color='red', size=15),
                            name='Продажа')
figure_stop_loss = go.Scatter(x=report[report['signal'] == 'stop-loss']['date'],
                             y=report[report['signal'] == 'stop-loss']['share_price'],
                             mode='markers',
                             marker=dict(symbol='triangle-down', color='black', size=15),
                             name='Стоп-лосс')

#### Остается определить график позиций (1 - все средства в акциях, 0 - все средства в кэше)

In [33]:
# Нижнее окно: позиция
trace_position = go.Scatter(
    x=info['Date'],
    y=for_graph_down['position'],
    mode='lines',
    name='Позиция (где находится кэш)',
    line=dict(color='purple', width=2))

#### Создаем для начала "макет", где уместится два окна, задаем размеры

In [34]:
fig = make_subplots(
    rows=2, cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3], subplot_titles=('График OHLC и скользящих средних', 'График позиции'))

#### На этот макет добавляем наши графики

In [35]:
fig.add_trace(trace_ohlc, row=1, col=1)
fig.add_trace(line_short_window, row=1, col=1)
fig.add_trace(line_long_window, row=1, col=1)
fig.add_trace(figure_sig_buy, row=1, col=1)
fig.add_trace(figure_sig_sale, row=1, col=1)
fig.add_trace(figure_stop_loss, row=1, col=1)
fig.add_trace(trace_position, row=2, col=1)

fig;

#### Обновляем все размеры графика и параметры, потому что наезжает друг на друга

In [36]:
fig.update_layout(
    height=800,
    showlegend=True,
    margin=dict(t=40, b=80, l=50, r=50),
    xaxis=dict(rangeslider=dict(visible=False)),
    xaxis2=dict(rangeselector=dict(
            xanchor='left',
            yanchor='top')))

fig;

#### Осталось подписать оси

In [37]:
fig.update_yaxes(title_text="Цена", row=1, col=1)
fig.update_yaxes(title_text="Позиция", range=[-0.1, 1.1], row=2, col=1)
fig.update_xaxes(title_text="Дата", row=2, col=1)

fig.show()